Предполагается, что на предыдущих этапах мы получили какой-то список данных с PDB и 2 идентификаторами цепочек. Про одну из таких цепочек мы знаем, что она пришла от онкогена.

Ниже я хочу проверить, что загруженные из репозитория сохраненные файлы открываются корректно и они действительно большие (вчера у меня была проблема с их загрузкой из репозитория)

Предполагается, что ранее были запущены скрипты

```{bash}
make_gene_list.py

kegg.py
```

В репозитории файл сгенерирован питоном 3, поэтому его можно загрузить pickle по протоколу версии 3 (которая есть только в 3 питоне).

In [1]:
import pickle

In [2]:
data_file = "gene_name_to_gene_ids_pdb_ids.pickle"

In [3]:
genes = pickle.load(open(data_file, "rb"))

Посмотрим общее количество всего (все гены, для которых есть информация по структурам)

In [4]:
gene_candidates = list(filter(lambda x: x!='', list(genes)))
print("total", len(gene_candidates))
gene_candidates = list(filter(lambda x: x!='' and len(genes[x][1]) > 0, list(genes)))
print("with pdb information", len(gene_candidates))

total 2230
with pdb information 1164


Теперь хорошей идеей будет оставить только гены, в последовательности аминокислот которых есть цистеин (соответствует текущим условиям задачи, если я правильно понимаю).

Но сначала посмотрим, сколько идентификаторов путей и различных PDB файлов соответствуют найденным генам

In [5]:
totalGeneIds = set()
totalPdbIds = set()
for geneIds, pdbIds in genes.values():
    if len(pdbIds) == 0:
        continue
    totalGeneIds |= set(geneIds)
    totalPdbIds |= set(pdbIds)
    #break

In [6]:
print("total geneIds", len(totalGeneIds))
print("total pdbids", len(totalPdbIds))

total geneIds 4578
total pdbids 13461


Теперь для каждого из генов посмотрим, есть ли там цистеин и оставим те структуры, для которых он точно где-то есть (длительная операция)

In [7]:
from structure_processor import getKeggInfoForGene

@> Local PDB folder is set: '/Users/lacemaker/github/biohack-2017/pdb'
INFO:.prody:Local PDB folder is set: '/Users/lacemaker/github/biohack-2017/pdb'
@> A plain folder structure will be assumed.
INFO:.prody:A plain folder structure will be assumed.


In [17]:
filteredPdbByGeneIds = dict()
if os.path.exists("filtered_pdb_list.txt"):
    with open("filtered_pdb_list.txt") as f:
        for line in f:
            name, geneId, pdbIds = line.strip().split()
            if not name in filteredPdbByGeneIds:
                filteredPdbByGeneIds[name] = dict()
            filteredPdbByGeneIds[name][geneId] = pdbIds.split(",")
            pass
        
for gene_name in gene_candidates:
    if gene_name in filteredPdbByGeneIds:
        continue
    pdbIds, geneIdByPDB, uniprotIdsByGene, aaSequencesByGene, pdbIdsByGene = getKeggInfoForGene("hsa", gene_name)
    filteredPdbByGeneIds[gene_name] = {
        geneId: pdbIdsByGene[geneId] for geneId in aaSequencesByGene
        if geneId in pdbIdsByGene and aaSequencesByGene[geneId].find('C') >= 0 # it means that there is cysteine
    }
    with open("filtered_pdb_list.txt", 'a') as f:
        for geneId in filteredPdbByGeneIds[gene_name]:
            f.write(" ".join([gene_name, geneId, ",".join(pdbIdsByGene[geneId])
                             ])+"\n")

In [66]:
keggCache = pickle.load(open("keggToUniprot", "rb"))


In [81]:
set(keggCache[list(keggCache)[3]])

{'A0A024R4G8', 'B2RCA1', 'P52735', 'Q96BD6', 'Q96MN2'}

In [20]:
 from functools import reduce

In [77]:
reorderedAndFiltered = dict()
for gene_name in gene_candidates:
    pdbIds = reduce(lambda x, y:x|y, map(set, filteredPdbByGeneIds[gene_name].values()), set())
    genes = reduce(lambda x, y: x|y, [set(keggCache[k]) for k in filteredPdbByGeneIds[gene_name] if k in keggCache], set())
    reorderedAndFiltered[gene_name] = (
        genes
        , pdbIds)
    
with open("gene_name_to_gene_ids_pdb_ids_filtered_by_cys.pickle","wb") as f:
    pickle.dump(reorderedAndFiltered,f)  


In [83]:
reorderedAndFiltered;

In [84]:
len(list(filteredPdbByGeneIds))

filtered_candidates = list(filter(lambda x: x!='', list(filteredPdbByGeneIds)))
print("total", len(filtered_candidates))
filtered_candidates = list(filter(lambda x: x!='' and len(filteredPdbByGeneIds[x]) > 0, list(filteredPdbByGeneIds)))
print("with pdb information", len(filtered_candidates))

total 1164
with pdb information 1146


In [85]:
totalPdb = reduce(lambda x, y:x|y, 
[
    reduce(lambda x, y:x|y, map(set, filteredPdbByGeneIds[gene].values()) , set())
for gene in filteredPdbByGeneIds], set())
len(totalPdb)

13191

цистеин есть почти везде:)

In [90]:
from structure_processor import *

In [87]:
pdbIdsByGene

{'hsa:10133': ['5EOF', '5EOA', '5B83', '5AAZ', '2LO4'],
 'hsa:5764': ['2N6F'],
 'hsa:5771': ['1L8K'],
 'hsa:6375': ['2N54', '2JP1', '2HDM', '1J9O', '1J8I']}

In [49]:
orderedGeneNames=sorted([(
    sum(
        [len(filteredPdbByGeneIds[gene][geneId]) for geneId in filteredPdbByGeneIds[gene]])
, gene) 
for gene in filteredPdbByGeneIds ], reverse=True)


In [53]:
with open("gene_stats.csv", "w") as f:
    for (number, name) in orderedGeneNames:
        f.write(name +";"+str(number)+"\n")

In [88]:
filteredPdbByGeneIds["TP53"]

{'hsa:2593': ['3ORH'],
 'hsa:50484': ['4DJN', '3HF1', '2VUX'],
 'hsa:51002': ['3ENP'],
 'hsa:57103': ['3DCY'],
 'hsa:6288': ['4IP8', '4IP9'],
 'hsa:7157': ['3D06',
  '5G4N',
  '5AOK',
  '3ZME',
  '5AB9',
  '5G4M',
  '5A7B',
  '3D08',
  '5LAP',
  '4AGQ',
  '4IBY',
  '4AGO',
  '5AOJ',
  '5G4O',
  '5AOL',
  '4KVP',
  '4AGP',
  '1AIE',
  '4AGM',
  '2PCX',
  '4AGN',
  '2X0U',
  '2B3G',
  '5ABA',
  '2J1X',
  '2XWR',
  '2J1Y',
  '4IBT',
  '4IBU',
  '4AGL',
  '3IGK',
  '3D05',
  '1C26',
  '5AOM',
  '2WGX',
  '5AOI',
  '4IJT',
  '4IBS',
  '4IBW',
  '4IBQ',
  '2X0V',
  '3IGL',
  '3D0A',
  '2AC0',
  '5BUA',
  '4LOE',
  '2AHI',
  '2YBG',
  '3D09',
  '4HJE',
  '3KZ8',
  '4QO1',
  '4IBZ',
  '2BIM',
  '4LOF',
  '2OCJ',
  '4IBV',
  '2X0W',
  '3KMD',
  '3D07',
  '2ATA',
  '1YCS',
  '1TSR',
  '1TUP',
  '4XR8',
  '4LO9',
  '2ADY',
  '1KZY',
  '5LGY',
  '5ECG',
  '2H1L',
  '2MZD',
  '2RUK',
  '2MEJ',
  '2LY4',
  '2L14',
  '2K8F',
  '2VUK',
  '2J1W',
  '2J1Z',
  '2J20',
  '2J21',
  '2J0Z',
  '2GS0',
  '2FE